In [1]:
pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os import path
import os, sys
import time
import json
import datetime
import csv
from scipy.stats import pearsonr, spearmanr
import yfinance as yf
from transformers import pipeline

In [3]:
directory = "/Users/mortenlassen/Desktop/CBS - noter/Kandidat/3. semester/Business Data Processing and Business Intelligence/Eksamen/Txt"

file_path_liste = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        file_path_liste.append(f)

In [4]:
all_data = []

for i in range(len(file_path_liste)):
    json_file_path = file_path_liste[i]
    with open(json_file_path, 'r') as f:
        data = json.load(f)
        all_data.append(data)

In [5]:
all_dataframes = []        
for j in range(len(all_data)):
    data_frame = pd.DataFrame.from_dict(all_data[j])
    all_dataframes.append(data_frame)

In [6]:
df = pd.DataFrame()
for i in range(0,len(all_dataframes),1):
    data = all_dataframes[i]
    df = df.append(data)
df = df.reset_index()
df = df.drop('index', axis=1)
# Reorder the columns in place
df = df[['Date', 'Headline', 'Abstract']]

In [7]:
Headline_list = df["Headline"].tolist() 
Abstract_list = df["Abstract"].tolist()

In [8]:
columns = ['Abstract']
df_Abstract = pd.DataFrame(Abstract_list, columns=columns)
df_Abstract.fillna('', inplace=True)
Abstract_list = df_Abstract['Abstract'].tolist()

In [9]:
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment_headline_list = []
for i in range(len(Headline_list)):
    sentiment_headline = sentiment_pipeline(Headline_list[i])
    sentiment_headline_list.append(sentiment_headline)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [10]:
df_sentiment_headline = pd.DataFrame(columns=['label', 'score'])
for sublist in sentiment_headline_list:
    df_sentiment_headline = df_sentiment_headline.append(sublist[0], ignore_index=True)

In [11]:
# Original range
min_value = 0.5
max_value = 1.0

# Function to transform scores

def transform_score(score, label):
    if label == 'NEGATIVE':
        return -score  # Invert the score for negative labels
    else:
        return score

# Apply the transformation
df_sentiment_headline['Transformed Score'] = df_sentiment_headline.apply(lambda row: transform_score((row['score']- min_value)/(max_value - min_value), row['label']), axis=1)

In [12]:
# Define a function to categorize values
def categorize_value(value):
    if value <= -0.5:
        return 'Negative'
    elif -0.5 < value <= 0.5:
        return 'Neutral'
    else:
        return 'Positive'
    
df_sentiment_headline['Final Label'] = df_sentiment_headline['Transformed Score'].apply(categorize_value)

In [13]:
df_sentiment_headline.rename(columns={'label': 'Headline label', 'score': 'Headline score','Transformed Score': 'Headline Transformed Score', 'Final Label': 'Headline Final Label'}, inplace=True)

In [14]:
sentiment_abstract_list = []
for i in range(len(Abstract_list)):
    sentiment_abstract = sentiment_pipeline(Abstract_list[i])
    sentiment_abstract_list.append(sentiment_abstract)

In [15]:
df_sentiment_abstract = pd.DataFrame(columns=['label', 'score'])
for sublist in sentiment_abstract_list:
    df_sentiment_abstract = df_sentiment_abstract.append(sublist[0], ignore_index=True)

In [16]:
# Original range
min_value = 0.5
max_value = 1.0

# Function to transform scores
def transform_score(score, label):
    if label == 'NEGATIVE':
        return -score  # Invert the score for negative labels
    else:
        return score

# Apply the transformation
df_sentiment_abstract['Transformed Score'] = df_sentiment_abstract.apply(lambda row: transform_score((row['score']- min_value)/(max_value - min_value), row['label']), axis=1)

In [17]:
# Define a function to categorize values
def categorize_value(value):
    if value <= -0.5:
        return 'Negative'
    elif -0.5 < value <= 0.5:
        return 'Neutral'
    else:
        return 'Positive'
    
df_sentiment_abstract['Final Label'] = df_sentiment_abstract['Transformed Score'].apply(categorize_value)

In [18]:
values_to_replace = [0.7481210231781006]
df_sentiment_abstract.replace(values_to_replace, np.nan, inplace=True)
df_sentiment_abstract.fillna('NaN', inplace=True)
# Change values in 'Value' column based on 'Category' column
df_sentiment_abstract.loc[df_sentiment_abstract['score'] == 'NaN', 'Transformed Score'] = 'NaN'
df_sentiment_abstract.loc[df_sentiment_abstract['score'] == 'NaN', 'Final Label'] = 'NaN'

In [19]:
df_sentiment_abstract.rename(columns={'label': 'Abstract label', 'score': 'Abstract score','Transformed Score': 'Abstract Transformed Score', 'Final Label': 'Abstract Final Label'}, inplace=True)

In [20]:
Final_df = df.reset_index(drop=True)

In [21]:
merged_df = pd.concat([Final_df,df_sentiment_headline],axis = 1)
final_merged_df = pd.concat([merged_df,df_sentiment_abstract],axis=1)
final_merged_df.columns = ['Date','Headline','Abstract','Headline label','Headline score','Headline Transformed Score', 'Headline Final Label','Abstract label','Abstract score','Abstract Transformed Score', 'Abstract Final Label']

In [22]:
final_merged_df

,Date,Headline,Abstract,Headline label,Headline score,Headline Transformed Score,Headline Final Label,Abstract label,Abstract score,Abstract Transformed Score,Abstract Final Label
0,2019-12-30,Tesla delivers first China-made cars from $5bn...,NaN,POSITIVE,0.998886,0.997772,Positive,POSITIVE,NaN,NaN,NaN
1,2019-11-22,Tesla to unveil long-hyped 'cyberpunk' electri...,NaN,NEGATIVE,0.998076,-0.996152,Negative,POSITIVE,NaN,NaN,NaN
2,2019-11-22,Cybertruck: Tesla unveils new pickup truck but...,NaN,NEGATIVE,0.991762,-0.983523,Negative,POSITIVE,NaN,NaN,NaN
3,2019-11-13,Tesla cites Brexit as Germany chosen over UK f...,NaN,NEGATIVE,0.978572,-0.957144,Negative,POSITIVE,NaN,NaN,NaN
4,2019-09-10,Video appears to show Tesla driver asleep at t...,NaN,NEGATIVE,0.999419,-0.998839,Negative,POSITIVE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6600,2023-07-28,Birmingham says opposition doesn’t ‘fear’ earl...,NaN,POSITIVE,0.804263,0.608526,Positive,POSITIVE,NaN,NaN,NaN
6601,2023-02-09,Deliveroo cutting 9% of staff; Bank of England...,NaN,NEGATIVE,0.994698,-0.989396,Negative,POSITIVE,NaN,NaN,NaN
6602,2023-08-23,Two charged over building fire – as it happened,NaN,POSITIVE,0.946528,0.893057,Positive,POSITIVE,NaN,NaN,NaN
6603,2023-01-13,New federal court rules over access to documen...,NaN,NEGATIVE,0.998906,-0.997812,Negative,POSITIVE,NaN,NaN,NaN


In [23]:
column_to_remove = ['Headline label','Headline score','Abstract label','Abstract score']
final_merged_df= final_merged_df.drop(column_to_remove,axis=1)

In [24]:
# Convert the 'Abstract Transformed Score' to numeric, errors='coerce' will replace non-numeric values with NaN
final_merged_df['Abstract Transformed Score'] = pd.to_numeric(final_merged_df['Abstract Transformed Score'], errors='coerce')

In [25]:
average_by_category_headline = final_merged_df.groupby('Date')['Headline Transformed Score'].mean()
average_by_category_abstract = final_merged_df.groupby('Date')['Abstract Transformed Score'].mean()
Headline_score_average = pd.DataFrame(average_by_category_headline)
Abstract_score_average = pd.DataFrame(average_by_category_abstract)

## Getting data from Yahoo Finance

In [26]:
# Ticker symbol for Tesla Inc.
ticker_symbol = 'TSLA'

# Set the start and end dates
start_date = '2012-01-01'
end_date = '2023-11-28'

# Download historical stock data
data = yf.download(ticker_symbol, start=start_date, end=end_date, freq='B')


[*********************100%***********************]  1 of 1 completed


In [27]:
Tesla_df=data.drop(['Open','High','Low','Adj Close'],axis=1)

In [28]:
Tesla_df.index = Tesla_df.index.date

In [29]:
Tesla_df = Tesla_df.rename_axis('Date')

In [30]:
Tesla_df_reset = Tesla_df.reset_index()
Tesla_df_reset['Date'] = pd.to_datetime(Tesla_df_reset['Date'])
Headline_score_average_reset = Headline_score_average.reset_index()
Headline_score_average_reset['Date'] = pd.to_datetime(Headline_score_average_reset['Date'])

In [31]:
merged_df_all_closing_prices_headline = pd.merge(Tesla_df_reset, Headline_score_average_reset, on='Date', how='outer')
merged_df_only_headline_score = pd.merge(Tesla_df_reset, Headline_score_average_reset, on='Date', how='right')

In [32]:
columns_to_fill_na = ['Headline Transformed Score']
merged_df_all_closing_prices_headline[columns_to_fill_na] = merged_df_all_closing_prices_headline[columns_to_fill_na].fillna(0)
merged_df_only_headline_score.fillna(method='ffill', inplace=True)

In [33]:
merged_df_all_closing_prices_headline.fillna(method='ffill', inplace=True)

## Correlation Analysis - headline

In [34]:
df_scores_data_headline = merged_df_only_headline_score
df_closing_data_headline = merged_df_all_closing_prices_headline

In [36]:
Change_in_scores_headline_closing_data = df_scores_data_headline['Close'] - df_scores_data_headline['Close'].shift(1)
df_scores_data_headline['Change'] = Change_in_scores_headline_closing_data
df_scores_data_headline = df_scores_data_headline.fillna(0)

In [37]:
# Pearson correlation
pearson_corr_price_headline, _ = pearsonr(df_scores_data_headline['Change'], df_scores_data_headline['Headline Transformed Score'])

In [38]:
print(f"Pearson Correlation: {pearson_corr_price_headline}")

Pearson Correlation: 0.013078529872870325


In [40]:
df_monthly_articles = final_merged_df

# Convert the 'Date' column to a pandas datetime object
df_monthly_articles['Date'] = pd.to_datetime(df_monthly_articles['Date'])

# Create a PeriodIndex with monthly frequency
df_monthly_articles['Month'] = df_monthly_articles['Date'].dt.to_period('m')

# Group by the 'month' and count the number of values
monthly_counts_headline = df_monthly_articles['Headline'].groupby(df_monthly_articles['Month']).size().reset_index(name='Count')

In [41]:
df_monthly_volume = Tesla_df_reset

# Convert the 'Date' column to a pandas datetime object
df_monthly_volume['Date'] = pd.to_datetime(df_monthly_volume['Date'])

# Create a PeriodIndex with monthly frequency
df_monthly_volume['Month'] = df_monthly_volume['Date'].dt.to_period('m')

# Group by the 'month' and count the number of values
monthly_counts_volume = df_monthly_volume['Volume'].groupby(df_monthly_volume['Month']).sum().reset_index(name='Sum')

In [42]:
# Pearson correlation
pearson_corr_Volume_headline, _ = pearsonr(monthly_counts_volume['Sum'], monthly_counts_headline['Count'])

In [43]:
print(f"Pearson Correlation: {pearson_corr_Volume_headline}")

Pearson Correlation: 0.27658081258042533


## Correlation Analysis - Abstract

In [44]:
Tesla_df_reset = Tesla_df.reset_index()
Tesla_df_reset['Date'] = pd.to_datetime(Tesla_df_reset['Date'])
Abstract_score_average_reset = Abstract_score_average.reset_index()
Abstract_score_average_reset['Date'] = pd.to_datetime(Abstract_score_average_reset['Date'])

In [45]:
merged_df_all_closing_prices_abstract = pd.merge(Tesla_df_reset, Abstract_score_average_reset, on='Date', how='outer')
merged_df_only_abstract_score = pd.merge(Tesla_df_reset, Abstract_score_average_reset, on='Date', how='right')

In [46]:
columns_to_fill_na_abstract = ['Abstract Transformed Score']
merged_df_all_closing_prices_abstract[columns_to_fill_na_abstract] = merged_df_all_closing_prices_abstract[columns_to_fill_na_abstract].fillna(0)
merged_df_only_abstract_score.fillna(method='ffill', inplace=True)

In [47]:
merged_df_all_closing_prices_abstract.fillna(method='ffill', inplace=True)

In [48]:
df_scores_data_abstract = merged_df_only_abstract_score
df_closing_data_abstract = merged_df_all_closing_prices_abstract

In [49]:
Change_in_closing_price_score_data_abstract = df_scores_data_abstract['Close'] - df_scores_data_abstract['Close'].shift(1)
df_scores_data_abstract['Change'] = Change_in_closing_price_score_data_abstract
df_scores_data_abstract = df_scores_data_abstract.fillna(0)

In [50]:
# Pearson correlation
pearson_corr_price_abstract, _ = pearsonr(df_scores_data_abstract['Change'], df_scores_data_abstract['Abstract Transformed Score'])

In [51]:
print(f"Pearson Correlation: {pearson_corr_price_abstract}")

Pearson Correlation: 0.010047570072812064


In [52]:
df_monthly_abstract = final_merged_df

# Convert the 'Date' column to a pandas datetime object
df_monthly_abstract['Date'] = pd.to_datetime(df_monthly_abstract['Date'])

# Create a PeriodIndex with monthly frequency
df_monthly_abstract['Month'] = df_monthly_abstract['Date'].dt.to_period('m')

# Count the number of observations where the text is longer than 0 characters on a quarterly basis
monthly_counts_abstract = (df_monthly_abstract[df_monthly_abstract['Abstract'].str.len() > 0].groupby('Month').size().reset_index(name='Count'))

In [53]:
df_monthly_volume = Tesla_df_reset

# Convert the 'Date' column to a pandas datetime object
df_monthly_volume['Date'] = pd.to_datetime(df_monthly_volume['Date'])

# Create a PeriodIndex with monthly frequency
df_monthly_volume['Month'] = df_monthly_volume['Date'].dt.to_period('m')

# Group by the 'Month' and count the number of values
monthly_counts_volume = df_monthly_volume['Volume'].groupby(df_monthly_volume['Month']).sum().reset_index(name='Sum')


In [54]:
# Pearson correlation
pearson_corr_Volume_abstract, _ = pearsonr(monthly_counts_volume['Sum'], monthly_counts_abstract['Count'])

In [55]:
print(f"Pearson Correlation: {pearson_corr_Volume_abstract}")

Pearson Correlation: 0.32172928705292647
